### Sampling based on SemEval + keywords extraction

In [928]:
import pandas as pd
import configparser
from newspaper import Article, Config
import time
import random
import requests
import json
import re
import deepl

config = configparser.ConfigParser()
config.read("keys.config")

hf_key = config['credentials']['hf_key']
openai_key = config['credentials']['openai_key']
key_deepL = config['credentials']['deepl_key']

In [61]:
classes = {4: 'Very dissimilar', 3: 'Somewhat dissimilar', 2: 'Somewhat similar', 1: 'Very similar', 0: '-'}

data = pd.read_csv("final_evaluation_data.csv")

data = data[(data.url1_lang == 'ru') | (data.url1_lang == 'pl') | (data.url1_lang == 'en')]
data["Overall_Class"] = data['Overall'].apply(lambda x: round(x)).apply(lambda x: classes[x])

VS = data[data.Overall_Class == 'Very similar'].sample(n=175)
SS = data[data.Overall_Class == 'Somewhat similar'].sample(n=175)
SD = data[data.Overall_Class == 'Somewhat dissimilar'].sample(n=175)
VD = data[data.Overall_Class == 'Very dissimilar'].sample(n=175)

sample = pd.concat([VS, SS, SD, VD], ignore_index=True)

### Newspaper3k

In [234]:
pattern_mistake = r"Article `download\(\)` failed with 403 Client Error: Forbidden for url: (.*) on URL .*"

def download_newspaper_3l(link, lang, max_retries=5, set_config=False):
    config = Config()
    if set_config:
        config.headers = { #https://iplogger.org/useragents/
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0',
            'Accept-Language': 'uk-UA,uk;q=0.8',
            'Accept': 'test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Referer': 'https://www.google.com/',
            'DNT': '1',
            'Connection': 'keep-alive'
        }
    if link != "Not found":
        for attempt in range(max_retries):
            try:
                config.request_timeout = random.randint(5, 14)
                article = Article(link, language=lang, config=config)
                article.download()
                article.parse()
                article.nlp()
                print("success")
                print(" ".join(article.keywords))
                return article.url, article.text, article.publish_date, article.summary, " ".join(article.keywords)
            except Exception as e:
                print(e)
                if not set_config:
                    match = re.match(pattern_mistake, str(e))
                    if match: #403 
                        url = match.group(1)
                        print('Now instead trying with url:', url)
                        url, text, date, summary, keywords = download_newspaper_3l(url, lang, 5, True)
                        return url, text, date, summary, keywords
                print(f"Attempt {attempt+1} of {max_retries}")
                sleep_time = 2 ** attempt + 10
                print(f"Retrying in {sleep_time} seconds.")
                time.sleep(sleep_time)
    else:
        print('Not found')
        return "","","","",""
    print("fail")
    return "","","","",""

In [ ]:
def try_both_links(link1, link2, lang):
    url, text, publish_date, summary, keywords = download_newspaper_3l(link1, lang)
    if text == "":
        print('Archive failed')
        url, text, publish_date, summary, keywords = download_newspaper_3l(link2, lang)
        if text == "":
            print('Main link also failed')
    return url, text, publish_date, summary, keywords

new_columns1 = sample.apply(lambda row: try_both_links(row.ia_link1, row.link1, row.url1_lang),
                                                      axis=1, result_type='expand')
new_columns1.columns = ['source1', 'text1', 'date1', 'extractive_summary1', 'keywords1']

sample = pd.concat([new_columns1, sample], axis=1)


new_columns2 = sample.apply(lambda row: try_both_links(row.ia_link2, row.link2, row.url2_lang),
                                                      axis=1, result_type='expand')
new_columns2.columns = ['source2', 'text2', 'date2', 'extractive_summary2', 'keywords2']
sample = pd.concat([new_columns2, sample], axis=1)

In [46]:
sample = sample[(sample.text1 != "") & (sample.text2 != "")]
VS = sample[sample.Overall_Class == 'Very similar'].sample(n=160)
SS = sample[sample.Overall_Class == 'Somewhat similar'].sample(n=160)
SD = sample[sample.Overall_Class == 'Somewhat dissimilar'].sample(n=160)
VD = sample[sample.Overall_Class == 'Very dissimilar'].sample(n=160)

sample = pd.concat([VS, SS, SD, VD], ignore_index=True)

In [47]:
keywords_adjusting = lambda cell: ";".join(cell.split(" "))

sample['keywords1'] = sample['keywords1'].apply(keywords_adjusting)
sample['keywords2'] = sample['keywords2'].apply(keywords_adjusting)

### Keywords

- https://huggingface.co/docs/api-inference/detailed_parameters
- https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2
- https://www.promptingguide.ai/models/mistral-7b
- https://docs.mistral.ai/guides/prompting-capabilities/

In [48]:
news_example = "В Киеве 13 марта внезапно изменится погода. В столице Украины поднимется сильный ветер.\n\nОб этом написала пресс служба ГУ ГСЧС Украины в г. Киеве со ссылкой на Укргидрометцентр. В городе объявили І уровень опасности – желтый.\n\n\"В Киеве в ближайший час и до конца суток 13 марта порывы ветра 15-18 м/с\", – говорится в сообщении.\n\nВ Киеве резко изменится погода. Иллюстрация\n\nКак сообщал OBOZREVATEL, в Киев в марте придет настоящая весна – потеплеет, вместо снега пойдет разве что дождь, чаще будет солнечная погода.\n\nНе надоедаем! Только самое важное - подписывайся на наш Telegram-канал"
mistral_prompt = lambda cell: "<s>[INST] Extract 10 keywords from a news article. The news article: \\" + news_example + "\\ Order them from the most informative to the least, in the following format: 1. <top-1 keyword>; 2. <top-2 keyword>; ... ;  10. <top-10 keyword>; [/INST] 1. погода; 2. Украина; 3. опасность; 4. Киев; 5. ветер; 6. Укргидрометцентр; 7. март; 8. изменится; 9. весна; 10. внезапно; </s> [INST] Extract 10 keywords from a news article. The news article: \\" + cell + "\\ Order them from the most informative to the least, in the following format: 1. <top-1 keyword>; 2. <top-2 keyword>; ... ;  10. <top-10 keyword>; [/INST]"

sample["mistral_prompt1"] = sample["text1"].apply(mistral_prompt)
sample["mistral_prompt2"] = sample["text2"].apply(mistral_prompt)

In [289]:
headers = {'Content-type': 'application/json', "Authorization": (f"Bearer " + hf_key)}

def query_mistral(prompt,
                  max_new_tokens=200,
                  do_sample=False, 
                  temperature=0.001,
                  top_p=0.999,
                  return_full_text=False):
    parameters = {'max_new_tokens':max_new_tokens, 
                  'return_full_text': return_full_text, 
                  'do_sample': True, 
                  'top_p':top_p, 
                  'temperature':temperature}
    options = {'use_cache': False}
    payload = {'inputs': prompt,
               'parameters': parameters,
               'options': options}
    data = json.dumps(payload)
    response = requests.request("POST",
                                "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2",
                                headers=headers,
                                data=data)
    try:
        return json.loads(response.content.decode("utf-8"))[0]['generated_text']
    except Exception as e:
        print(json.loads(response.content.decode("utf-8")))
    return 'Model error'

In [52]:
sample["mistral_keywords1"] = sample["mistral_prompt1"].apply(query_mistral)

[{'generated_text': ' 1. World War II veteran; 2. coronavirus; 3. hospital; 4. guard of honor; 5. nurses; 6. Tickhill Road Hospital; 7. Doncaster; 8. COVID-19; 9. recovery; 10. 99-year-old.'}]
[{'generated_text': ' 1. Parlament Europejski; 2. Polska; 3. praworządności; 4. rezolucja; 5. Europarlament; 6. Węgry; 7. sytuacja; 8. sędziów; 9. procedura; 10. Unia Europejska;\n\n1. European Parliament; 2. Poland; 3. resolution; 4. righteousness; 5. Europarliament; 6. Hungary; 7. situation; 8. judges; 9. procedure; 10. European Union.'}]
[{'generated_text': ' 1. przebudowa; 2. ul. Bohaterów Getta Warszawskiego; 3. Elbud Szczecin; 4. umowa; 5. realizacja; 6. Zarząd Dróg i Transportu Miejskiego; 7. remont; 8. nawierzchnia; 9. zieleń; 10. inwestycja;\n\n1. Elbud Szczecin wins bid for road reconstruction in Szczecin city center;\n2. Ul. Bohaterów Getta Warszawskiego, Królowej Jadwigi, and Ściegiennego to be reconstructed;\n3. New contract for Elbud Szczecin in the ongoing road improvement project;

[{'generated_text': ' 1. парламентские выборы; 2. Израиль; 3. Биньямин Нетаньяху; 4. Ликуд; 5. победа; 6. Кнессет; 7. Правительственная коалиция; 8. Бени Гантц; 9. Кахоль-Лаван; 10. выборы;\n\n1. Израиль; 2. парламентские выборы; 3. Биньямин Нетаньяху; 4. Ликуд; 5. победа; 6. Кнессет; 7. Правительственная коалиция; 8. Бени Гантц; 9. Кахоль-Лаван; 10. выборы; 11. голова; 1'}]
[{'generated_text': " 1. wyborcy; 2. wyborcza; 3. komisja; 4. prezydenckie; 5. kandydaci; 6. podpisy; 7. zmarli; 8. PKW; 9. prokuratura; 10. wypadki nieprawidłowe;\n\nExplanation:\n\n1. wyborcy (voters)\n2. komisja (commission)\n3. prezydenckie (presidential)\n4. kandydaci (candidates)\n5. podpisy (signatures)\n6. zmarli (dead people)\n7. PKW (State Electoral Commission)\n8. prokuratura (prosecutor's office)\n9. wypadki nieprawidłowe (irregularities)\n"}]
[{'generated_text': " 1. coronavirus; 2. New York City; 3. first case; 4. woman; 5. Iran; 6. self-quarantined; 7. respiratory symptoms; 8. New York's Wadsworth La

[{'generated_text': " 1. Star Wars: The Clone Wars; 2. Disney+; 3. premiere; 4. Dave Filoni; 5. Anakin Skywalker; 6. Clone Wars; 7. prequel trilogy; 8. animated series; 9. Netflix; 10. season seven.\n\nAdditional keywords:\n\n11. Obi-Wan Kenobi\n12. Padmé Amidala\n13. Jedi\n14. Republic\n15. Separatists\n16. Animated episodes\n17. Ahsoka Tano\n18. Jedi Council\n19. Clones\n20. Droids\n21. Star Wars: The Last Jedi\n22. Luke Skywalker\n23. Disney's streaming service\n24. Critical reception\n25. Fan support."}]
[{'generated_text': ' 1. Manowska; 2. Sąd Najwyższy; 3. plan; 4. problemy; 5. odebranie; 6. przywilejów; 7. Sąd Najwyższy (SN); 8. Konstytucja; 9. zasady; 10. pracownicy\n\nor\n\n1. Manowska; 2. Sąd Najwyższy; 3. plan; 4. problems; 5. odebranie; 6. niezasadnych przywilejów; 7. SN; 8. Konstytucja; 9. zasady; 10. pracownicy, sędziowie.'}]
[{'generated_text': ' 1. Kinga Rusin; 2. Wiadomości; 3. Maciej Sawicki; 4. TVP; 5. relacja; 6. skrytykowano; 7. niedawna impreza; 8. pozwy; 9. mani

[{'generated_text': ' 1. Абхазия; 2. выборы; 3. президентские; 4. Аслан Бжания; 5. победа; 6. Центральная избирательная комиссия; 7. март, 23; 8. Итоги; 9. Рауль Хаджимба; 10. Верховный суд;\n\n1. Абхазия;\n2. выборы;\n3. президентские;\n4. Аслан Бжания;\n5. победа;\n6. Центральная избирательная комиссия;\n7. март, 23;\n8. итоги;\n9. Рауль Хаджимба;\n10. Верховный суд;\n11. презид'}]
[{'generated_text': ' 1. hostage; 2. India; 3. Uttar Pradesh; 4. armed man; 5. Subhash Batham; 6. birthday party; 7. police action; 8. villagers; 9. murder case; 10. Farukkhabad;\n\n1. Children; 11. house; 12. rescue; 13. gunshots; 14. crude bomb; 15. authorities; 16. criminal charges; 17. shoot-out; 18. injuries; 19. stand-off; 20. security personnel;\n\n21. murder; 22. framed; 23. pressurise; 24. wife; 25. killed; 26. beaten; 27. stones; 28. attempted; 29. scene'}]
[{'generated_text': ' 1. influenza; 2. antigenic imprinting; 3. flu season; 4. H1N1; 5. H3N2; 6. McMaster University; 7. susceptibility; 8. a

[{'generated_text': ' 1. $1,200 payment; 2. U.S. government; 3. COVID-19 pandemic; 4. rescue package; 5. eligibility; 6. adjusted gross income; 7. accessing funds; 8. IRS; 9. tax return; 10. economic impact payment.\n\nExplanation: The article primarily focuses on the upcoming $1,200 payment from the U.S. government, which is part of a larger rescue package. The article discusses the eligibility criteria for receiving the payment and how to access it. The article also mentions the COVID-19 pandemic and the impact it has had on the economy. The IRS and tax returns are mentioned in the context of accessing the payment.'}]
[{'generated_text': " 1. Монголия; 2. Великая Отечественная война; 3. Фашистская Германия; 4. Декларация; 5. Советский Союз; 6. Помощь; 7. Монгольская Народная Республика; 8. Халхин-Гол; 9. Поставки; 10. Советская армия.\n\nExplanation:\n\n1. Mongolia is the main keyword in the article as it is the subject of the news.\n2. The Great Patriotic War (Великая Отечественная 

...

[{'generated_text': ' 1. Чечня; 2. памятник; 3. медицинские работники; 4. погибшие; 5. Великая Отечественная война; 6. контртеррористическая операция; 7. установлен; 8. Республиканская клиническая больница; 9. Эпендиева; 10. оргкомитет;\n\n1. Чечня; 2. памятник; 3. медицинские работники; 4. погибшие; 5. Великая Отечественная война; 6. контртеррористическая операция; 7. увековечивание; 8. Республиканская клиническая больница; 9. Эпенди'}]
[{'generated_text': ' 1. Ани Лорак; 2. украинская певица; 3. реклама; 4. маски; 5. сториз; 6. фото; 7. критика; 8. Алла Крутая; 9. вечеринка; 10. Лев Лещенко;\n\n1. Ани Лорак; 2. певица; 3. реклама; 4. маска; 5. сториз; 6. фото; 7. удивила; 8. поклонники; 9. Алла Крутая; 10. вечеринка;\n\n1. Ани Лорак; 2. певица; 3. реклама; 4. маска; 5. сториз; 6. фото; '}]
[{'generated_text': ' 1. Татария; 2. строительство; 3. республиканская клиническая инфекционная больница; 4. корпус; 5. Кизань; 6. республиканский бюджет; 7. нового корпуса; 8. койки; 9. реанимацио

[{'generated_text': ' 1. цены; 2. вторичные квартиры; 3. российские города; 4. увеличение; 5. стоимость; 6. ипотечные ставки; 7. активизация; 8. покупатели; 9. инвесторы; 10. регионы\n\nExplanation: The article discusses the increase in prices for second-hand apartments in Russian cities with populations over 300,000 people. The keywords reflect the main topics of the article, including the increase in prices, second-hand apartments, Russian cities, and the reasons for the price rise, such as activating buyers and investors.'}]
[{'generated_text': ' 1. COVID-19; 2. Poland; 3. Mariusz Kamiński; 4. Ministerstwo Spraw Wewnętrznych i Administracji; 5. organizatorzy imprez masowych; 6. decyzje; 7. mecze piłkarskie; 8. PKO Bank Polski Ekstraklasy; 9. zakaz; 10. organizowane;\n\n1. COVID-19 outbreak; 2. Poland government; 3. Mariusz Kamiński (Interior Minister); 4. Decisions on mass events; 5. Organizers; 6. Football matches; 7. PKO Bank Polski Ekstraliga; 8. Ban; 9. No public attendance; 10.

[{'generated_text': ' 1. Челябинская область; 2. ФСБ России; 3. Анаболические препараты; 4. Пограничники; 5. Пункт пропуска "Бугристое"; 6. Обнаружен; 7. Крупная партия; 8. Документы; 9. Запасные части; 10. Компьютерная техника.'}]
[{'generated_text': " 1. zakażeniya; 2. Polska; 3. Ministerstwo Zdrowia; 4. pandemia; 5. koronawirus; 6. liczba zarażonych; 7. rośnie; 8. 1000 osób; 9. Ministerstwo Zdrowia (Poland); 10. SARS-CoV-19.\n\nExplanation: The article is about the increasing number of coronavirus cases in Poland, with the number of infected people exceeding 1000. The keywords reflect the main topics of the article, including the country (Polska), the ministry of health (Ministerstwo Zdrowia), the virus (koronawirus), the number of infected people (liczba zarażonych), and the ministry's warnings (Ministerstwo Zd"}]
[{'generated_text': ' 1. валюты; 2. Узбекистан; 3. курсы; 4. доллар; 5. евро; 6. Центральный банк; 7. рубль; 8. Ташкент; 9. рост; 10. июня; \n\nExplanation: The article i

[{'generated_text': " 1. курс доллара; 2. Украина; 3. гривня; 4. НБУ; 5. Минфин; 6. Владимир Мазуренко; 7. пороговая цифра; 8. инфляция; 9. МВФ; 10. прогноз.\n\nExplanation: The article discusses the potential increase in the dollar's value in Ukraine, reaching a psychological threshold of 25 UAH/$ by the end of February. The expert, Vladimir Mazurenko, explains the reasons for this trend, including import gas prices, low season for major exporters, and high demand for the dollar. The article also mentions the impact of this situation on inflation and the budget, as well as the MVF's predictions for the value of the hryvnia in the coming years."}]
[{'generated_text': ' 1. fire; 2. story; 3. colors; 4. remission; 5. old tree; 6. vortex; 7. elision; 8. realities; 9. human world; 10. bones.\n\nNote: This text is a poem and not a news article, but I have tried to extract the most relevant keywords based on the given context.'}]
[{'generated_text': " 1. ecommerce industry; 2. India; 3. regu

[{'generated_text': ' 1. Polska; 2. UE; 3. Brexit; 4. Nigel Farage; 5. Europejski parlament; 6. Polexit; 7. decyzje; 8. system rządowy; 9. Brukseli; 10. Radosław Sikorski.\n\nExplanation: The article discusses Nigel Farage\'s statement that Poland may be the next country to leave the EU, a potential "Polexit," and the reactions to this statement from various sources. The most informative keywords are those directly related to the topic of the article.'}]
[{'generated_text': ' 1. Василий Белоусов; 2. Мурманская область; 3. книга; 4. журналист; 5. второе издание; 6. ТАСС; 7. Почетный гражданин; 8. Мурманское Заполярье; 9. изданию; 10. встреча;\n\n1. Василий Белоусов; 2. книга; 3. Мурманская область; 4. журналист; 5. второе издание; 6. ТАСС; 7. Почетный гражданин; 8. Мурманское Заполярье; 9. изданию; 10. встреча с Виктором Сайгиным;\n\n1. Василий Белоусов; 2. книга; 3'}]
[{'generated_text': ' 1. Россия; 2. Роспотребнадзор; 3. коронавирус; 4. показатели; 5. заражение; 6. лаборатории; 7. те

In [ ]:
sample["mistral_keywords2"] = sample["mistral_prompt2"].apply(query_mistral)

In [54]:
pattern_mistral = r"^(1|2|3|4|5|6|7|8|9|(10)){1}\.\s*(.*)"

def answer_engineering_mistral(cell):
    new_cell = cell.split('\n\n')[0].strip()
    new_list = []
    for word in new_cell.split(';'):
        match = re.match(pattern_mistral, word.strip(' \n.'))
        if match:
            new_list.append(match.group(3).strip('.'))
    return ";".join(new_list)

sample["mistral_keywords1"] = sample["mistral_keywords1"].apply(answer_engineering_mistral)
sample["mistral_keywords2"] = sample["mistral_keywords2"].apply(answer_engineering_mistral)

In [953]:
def translate_to_ukr(keywords, language, target_lang='uk'):
    try:
        translator = deepl.Translator(key_deepL)
        result = translator.translate_text(keywords, source_lang=language, target_lang=target_lang)
        print(str(result))
        return str(result)
    except Exception as e:
        print(e)
    return keywords

In [65]:
sample["mistral_keywords1_translated"] = sample.apply(lambda row: translate_to_ukr(row.mistral_keywords1, row.url1_lang),
                                                      axis=1)

Ветеран Другої світової війни;коронавірус;лікарня;почесна варта;медсестри;лікарня Тікхілл Роуд;Донкастер;COVID-19;одужання;99-річний
Європейський парламент;Польща;верховенство права;резолюція;Європарламент;Угорщина;становище суддів;процедура;Європейський Союз
Реконструкція;вулиця Богатерова Ґетта Варшавського;Elbud Szczecin;контракт;реалізація;Міське управління доріг і транспорту;ремонт;дорожнє покриття;озеленення;інвестиції
ФАПи;Олександр Котов;Стругокрасненський район;оглянув;нове;медичне обслуговування;села;Молоді, Лудоні, Рівне;водогін;участь
Заняття;перенесено;березень;Школи;Матеуш Моравецький;Прем'єр-міністр;Лукаш Шумовський;Міністр охорони здоров'я;Діти;Марлена Малонґ
Тверська вулиця;Москва;фестиваль "Кримська весна";пішохідна;програма;інтелектуальні ігри; "Що? Де? Коли?";BrainZona;майстер-класи;Тверська вулиця (пішохідна)
Канцлер;Ріші Сунак;самозайняті;£9 млрд;коронавірус;підтримка;£2 500;прибуток;податкова декларація;червень
помер;коронавірус;ускладнення;чоловік;лікарня Тихи;с

мир;глобальний світ;злагоди;толерантності;міжнаціональні відносини;конфлікти;Казахстан;міжетнічна злагода;Нурсултан Назарбаєв;Асамблея народу Казахстану
коронавірус;Росія;Путін;Михайло Мішустін;Тетяна Голікова;Анна Попова;Михайло Мурашко;Євген Зінічев;SARS;MERS
Португалія;інфекція;коронавірус;двоє пацієнтів;лікарні;Порту;новий вірус;Європа;міністр охорони здоров'я;Граса Фрейташ
вбивство;ненавмисне вбивство;Гектор;Ентоні Теббен;Елізабет П. Лінч;стрілянина;судмедексперт округу Хеннепін;домашнє насильство;округ Ренвілл;розслідування
COVID-19;світ;зараження;коронавірус;ранок;число;підтверджених випадків;світове;летальні;Укрінформ;коронавірус;світ;зараження;4,8 мільйона;летальні;Укрінформ;Worldometer;316 тисяч;померлі;число випадків
відкриття, виробництв, торгових, торгових центрів, готелів, епідеміологічних, критеріїв, економіст, роботодавців, Польщі, розморожування, економіки
Виплата 1200 доларів США;уряд США;пандемія COVID-19;пакет допомоги;право на отримання;скоригований валовий дохід;д

Кладовища;Уряд;рішення;Право і справедливість;Ярослав Качинський;адміністратор;в'їзд;права;Повазькі кладовища;процедури
оренда житла;Флорида;правила;місцеве самоврядування;уряд штату;виборні лідери;Сент-Армандс-Кі;Лонгбот-Кі;острів Анна-Марія;законодавство;острів Анна-Марія
Прем'єр-міністр;Аскар Мамін;Казахстан;Республіка;Нур-Султан;Торгівля;Промисловість;Аль-Куварі;Катар;Зустрічі
Анна Ведута;Олексій Венедіков;домагання;Еха Москви;звинувачення;редактор;2012 рік;вечеря;автомобіль;сексуальне harassment
Київ;батько;погода;зник;проведення пошуків;огорож allotment;Кwisy river;прокуратура;підозрюваний
Меган Маркл;принц Гаррі;Арчі;слова;улюблений;мама;тато;книга;собака;джерело
терористи;Шоп'ян;ЛеТ;зіткнення;Шабір Ахмад Малік;Кульгам;Амір Ахмад Дар;Вадіна Мелхура;вилікуваний;підготовлений PK
Конституційний трибунал;Президент ТК Юлія Пшиленбська;Ельжбета Вітек;президентські вибори;термін;Конституція;Виборчий кодекс;спікер Сейму;заява;Конституційний трибунал - провадження
Ломбардія;італійський р

Вибори;Польща;опозиція;правляча коаліція;президент;Дональд Туск;вибори;конституція;Ярослав Качинський;надзвичайний стан
COVID-19;Сіетл;соціальна дистанція;Вашингтон;губернатор Джей Інслі;наказ залишатися вдома;Інститут моделювання хвороб;округ Кінг;швидкість передачі;коронавірус
Конгрес;американський народ;законодавчий процес;Гамільтон;Капітолій США;Палата представників;дебати;президентство;Конституція;конкуруючі інтереси
Карлос Гон;Японія;біг;автомобілебудівний альянс;Renault - Nissan - Mitsubishi;Осака;швидкісний поїзд "Сінкансен";Сінагава;домашній арешт;Ліван
Санкт-Петербург;депутати;Законодавчі збори;дитячий омбудсмен;Світлана Агапітова;Анна Мітяніна;процедура;кандидати;рішення;парламентська комісія
Примас Вишинський;ізоляція;Ева Чачковська;Ясна Гура;беатифікація;цикл;віра;надія;любов;інтерновані
дані;ви;сайт;інформація;матеріал;реклама;обробка;конфіденційність;політика;згода;мета;файли cookie
інфекція;коронавірус;SARS-CoV-2;Малопольська;лабораторія;тестування;мешканці;госпіталізов

Байєльса;Верховний суд;вибори;обраний губернатор;АПК;порушення;рішення;президент Мухаммаду Бухарі;ІНЕК;НДП
роздрібний кредитний портфель;угорські банки;листопад;MNB;7,022 трлн форинтів;операції;переоцінки;роздрібні кредити;роздрібні депозити;чисті притоки/відтоки
Тіло Христове;процесії;церкви;святкування;душпастирство;здоров'я;послання;єпархії;учасники;SARS-CoV-2
Президентські. вибори. 2020. відбудуться. 28. червня,. кандидати. у. президенти. коротко. про. передвиборчі. програми. на. посаду. президента.
Павлас Адамович;смертельний напад;Гданськ;TVN;рік потому;фото;президент;Вугільний ринок;меморіальна дошка;трагічна смерть
коронавірус;пандемія;Всесвітня організація охорони здоров'я (ВООЗ);Маргарет Гарріс;Європа;США;спалах;зараження;тестування;вакцина
SpaceX;NASA;місія;запуск;ракета Falcon 9;погодні умови;скасовано;американська команда;російська;Міжнародна космічна станція
погода;Україна;температура;південь;сонячна;схід;дощі;град;північ;центр
іпотека;Асоціація компаній фінансового посер

Сербія;комендантська година;Александр Вучич;пандемія;вірус SARS-CoV-2;запроваджено;вихідні;заборона;збіговиська;заходи
Żabka Polska;Jolanta Bańczerowska;кадрова політика;2016;розвиток;сучасні;процеси рекрутингу;системи оплати праці та преміювання;змінена організаційна культура;відкрита та дружня
Нацизм;Третій Рейх;Гітлер;расизм;ідеологія;відродження;людство;антикомунізм;націонал-соціалізм;єврей
атомна електростанція;Рівненська;відключила;Енергоатом;енергоблок;автоматичний захист;циркуляційні насоси;подія;INES;відключений
Михайло Мішустін;Прем'єр-міністр;Леонід Зорін;сценарист;драматург;письменник;смерть;театр;кінематограф;твори
Ана Чері;фото;фітнес-модель;Instagram;вбрання;контент;карантин;шанувальники;розкішні форми;коментарі
Рада Федерації;Держдума;поправки;заборона;нікотиновмісні суміші;електронні сигарети;вейпи;кальяни;тютюнова продукція;Валерій Рязанський
Президентські вибори;Анджей Дуда;Рафал Тшасковський;Конфедерація;Артур Дзямбор;другий тур;ПіС;РО;виборці;вибір між гільйотиною 

опитування;поляки;виборці;парламентські;вибори;Шимон Головня;партія;Koalicja Obywatelska;Lewica;Polskie Stronnictwo Ludowe
Єкатеринбург;кримінальна справа;депутат;Костянтин Кисельов;погрози;поліція;ГУ МВС РФ;прокуратура;ст.119 КК РФ;погроза вбивством
економічна криза;коронавірус;капіталізм;Насім Талеб;смертність;ВООЗ;виробництво;пандемія;інституції;економіки
Росія;Президент;Дмитро Пєсков;Ідлібі;Сирія;загинули;військовослужбовці;Турецька республіка;безпека;кордон
Австрія;коронавірус;Італія;потяг;прикордонники;лихоманка;ТПВ;OeBB;Reuters
нафта;ОПЕК+;виробники;видобуток;угода;ціни;учасники;скорочення;зустріч;Ірак, Нігерія
Вартові Галактики;Кіновсесвіт Marvel;Netflix;Hulu;Amazon Prime;стрімінговий сервіс;Disney+;Вартові Галактики 2;фільми Marvel;прокат або купівля
Міністерство освіти;Ігор Карпенко;випускні вечори;навчальні заклади;проведення;республіканський "Останній дзвоник";Бобруйськ;атестати;адміністрація навчальних закладів;школи
Банк Росії;кредитні організації;банківські операції;безп

Вибори;уряд;місцева;адміністрація;конституційний;обов'язок;діяти;перший;термін;стан;катастрофи;конституційні;підстави;очікування;Ярослав Качинський
сказ;Крим;дикі тварини;лисиці;бродячі собаки;вакцинація;ветеринарна лікарня;Россільгоспнагляд;2020 рік;Судакське лісомисливське господарство
Росія;Єдина Росія;Крим;чиновники;блокада;погода;фотосесія;шуби;влада;історія
Коронавірус;Італія;пандемія;померлі;охорона здоров'я;служба;Італія;оголошення;Cesvi;Gofundme;лікарі
Польща;коронавірус;епідемія;Матеуш Моравецький;карантин;прем'єр-міністр;кількість заражених;перевірки;солдати;боротьба з коронавірусом
коронавірус;Італія;лікарі;померли;медична федерація;служба охорони здоров'я;Італія;інфіковані;смерть;реанімація
Ухань;епідемія;Китай;лікарні;Лі Кецян;коронавірус;візит;нові заклади;лікарні;центральний уряд
вірусологи;пандемія;COVID-19;правила;травневі свята;безпека;дача;соціальна дистанція;харчування;вірусолог
Якутськ;Вілюйський район;Якутія;кримінальна справа;вбивство;рецидивіст;морзі;центральна

Rossmann;OnlyBio;заправка;Косметика;миючі засоби;Виробництво;Лодзь;органічний;натуральний;автоматичний
Набережні Челни;Subaru;офіційний дилерський центр;ТрансТехСервіс;японська марка;автоцентр;нове обладнання;комплексне післяпродажне обслуговування;дилери Subaru;РФ
Excel;гіперпосилання;стовпець;перетягування;автоматичне заповнення;клітинки;MJN1001-MJN1100;папка;функція копіювання;наступна клітинка
Детройт Пістонс;НБА;сезон;рекорд;відновлення;травми;склад;молоді гравці;драфт;перспективи
автофіксація;порушення;ПДР;знизилося;Міністр;внутрішніх справ;Арсен Аваков;Київ;швидкість;дороги
погода;Україна;Крим;поліція;нарколабораторія;виробництво наркотиків;підпільна група;кримінальна справа;наркотики;Псковська область
твори;Gremi Media SA;авторське право;розповсюдження;частина;ціле;заборона;всі сфери використання;Інтернет;згода
енцефаліт;вірус хвороби Борна 1 (BoDV-1);Німеччина;люди;смерть;землерийки;неврологічне захворювання;смертельний;дослідники;The Lancet Infectious Diseases
Деві, штат Флор

погоня;крадіжка;Желєзногорськ;Красноярський край;смартфон;банківський рахунок;поліція;зловмисниця;гаджет;пароль
Мішонна;Ходячі мерці;10-й сезон;повернення;Данія Гуріра;Вергілій;зброя;Шептуни;Ізабель Керролл;перипетії
парад домашніх тварин;Національний центр кваліфікованої сестринської допомоги;Подяка персоналу;Центр здоров'я Доброго Пастиря;Cornerstone Assisted Living;кози;коти;собаки;хендлери;заходи
ігри;2020;дозволи;PlayStation 5;Xbox Series X;відставання;майстер;монетизація;Switch;нові
виробництво;кисневі вентилі;Дагестан;Каспійськ;завод "Дагдизель";кисень;медичне обладнання;конструкторська документація;пандемія;Володимир Васильєв
закохані;карантин;ситуація;відстань;переживати;любов;підсумків;страждають;кордони;вірус
Тольятті;Самарська область;університет;онлайн-навчання;нова коронавірусна інфекція;профілактика;пресслужба;Повідомляє;16 березня;Росдистант
Московська житлоінспекція;продовження строку;перепланування;Москва;дозволи;рішення;строку дії;Мій Дім Москва;будівельні роботи;вин

In [ ]:
sample["mistral_keywords2_translated"] = sample.apply(lambda row: translate_to_ukr(row.mistral_keywords2, row.url2_lang),
                                                      axis=1)

In [ ]:
sample.to_csv("sources.csv")